# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 12hSleeping whole brain C57/BL6 strain (B6) Expression values and Group 2 be 12hSD whole brain C57/BL6 strain (B6) Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_ZT12_B6_'   # define a prefix to add to column names (mB6ing indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9442.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 87)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 87)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='B6_S_ZT12.CEL'
sd_filt ='B6_SD_ZT12.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239894_B6_S_ZT12.CEL,GSM239895_B6_S_ZT12.CEL,GSM239896_B6_S_ZT12.CEL
Probesets,,,
1427138_at,6.22061,6.29163,6.50084
1425600_a_at,7.78686,7.63007,7.86330
1457168_at,4.71687,4.61412,4.41388
1450135_at,5.23675,5.13114,4.93243
1424014_at,7.95263,8.06863,8.04257


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239906_B6_SD_ZT12.CEL,GSM239908_B6_SD_ZT12.CEL
Probesets,,
1427138_at,6.62444,6.35281
1425600_a_at,7.16445,7.19261
1457168_at,4.48406,4.50306
1450135_at,4.87594,4.91572
1424014_at,8.13662,8.05076


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.150932
1425600_a_at   -0.581547
1457168_at     -0.088063
1450135_at     -0.204277
1424014_at      0.072413
Name: 430AV2_ZT12_B6_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at      0.928108
1425600_a_at   -5.948762
1457168_at     -0.698637
1450135_at     -1.605922
1424014_at      1.190786
Name: 430AV2_ZT12_B6_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      0.754088
1425600_a_at   -4.833369
1457168_at     -0.567642
1450135_at     -1.304811
1424014_at      0.967513
Name: 430AV2_ZT12_B6_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.919472
1425600_a_at    4.372110
1457168_at      0.882143
1450135_at      1.091232
1424014_at      0.975130
Name: 430AV2_ZT12_B6_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_ZT12_B6_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_ZT12_B6_Enrich,430AV2_ZT12_B6_poolStDev,430AV2_ZT12_B6_Cohens_d,430AV2_ZT12_B6_J,430AV2_ZT12_B6_Hedges_g,430AV2_ZT12_B6_Var_d,430AV2_ZT12_B6_Var_g,430AV2_ZT12_B6_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1426915_at,Dapk1,death associated protein kinase 1,ENSMUSG00000021559,69635,NaN,protein kinase cascade // protein amino acid p...,protein binding // protein kinase activity // ...,cytoplasm,NaN,"mmu-let-7(TargetScan), mmu-let-7a(RNAhybrid|Ta...",...,1.570041e-01,0.281130,0.346310,0.005603,61.807444,0.8125,50.218548,382.849345,252.740388,15.897811
1445758_at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.515333e-01,0.250227,0.181092,0.003096,58.491257,0.8125,47.524146,342.956047,226.404578,15.046746
1436240_at,NaN,NaN,NaN,NaN,Mm.461284,NaN,NaN,NaN,NaN,NaN,...,1.061952e-27,6.627417,0.394830,0.010047,39.298884,0.8125,31.930343,155.273560,102.504811,10.124466
1455599_at,Gfod1,glucose-fructose oxidoreductase domain contain...,ENSMUSG00000051335,328232,NaN,metabolic process // oxidation reduction,electron carrier activity // catalytic activit...,extracellular region,NaN,"mmu-let-7(TargetScan), mmu-miR-128(TargetScan)...",...,4.520434e-08,0.541807,0.525412,0.015184,34.603708,0.8125,28.115513,120.574994,79.598336,8.921790
1446767_at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.520543e-01,0.287757,0.124882,0.003730,33.478167,0.8125,27.201011,112.912102,74.539630,8.633634
1427310_at,Bptf,bromodomain PHD finger transcription factor,ENSMUSG00000040481,207165,NaN,negative regulation of transcription from RNA ...,transcription factor binding // protein bindin...,nucleus,NaN,"mmu-miR-1192(miRanda), mmu-miR-128(miRanda), m...",...,8.758306e-01,0.359820,0.410793,0.012477,32.923135,0.8125,26.750047,109.226613,72.106631,8.491562
1434218_at,C330019G07Rik,proline rich 14-like [Source:MGI Symbol;Acc:MG...,ENSMUSG00000054280,215476,NaN,NaN,NaN,NaN,NaN,"mmu-let-7(TargetScan), mmu-miR-1(miRanda), mmu...",...,1.521830e-03,0.542187,0.227917,0.007231,31.519687,0.8125,25.609746,100.182401,66.136038,8.132407
1434960_at,Taf9b,"TAF9B RNA polymerase II, TATA box binding prot...",ENSMUSG00000047242,407786,NaN,transcription // transcription initiation // r...,DNA binding,nucleus // transcription factor TFIID complex,NaN,"mmu-let-7(TargetScan), mmu-let-7a(miRanda), mm...",...,7.496329e-02,0.342090,0.353968,0.011371,31.130101,0.8125,25.293207,97.741654,64.524764,8.032731
1446019_at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.935394e-01,0.197673,0.264982,0.009264,28.602740,0.8125,23.239726,82.645008,54.558619,7.386381


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL,avg-AK_S_ZT6,log_fold-AK_S_ZT6_vs_AK_SD_ZT6,fold-AK_S_ZT6_vs_AK_SD_ZT6,rawp-AK_S_ZT6_vs_AK_SD_ZT6,adjp-AK_S_ZT6_vs_AK_SD_ZT6,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,...,ANOVA-adjp,largest fold,430AV2_ZT12_B6_Enrich,430AV2_ZT12_B6_poolStDev,430AV2_ZT12_B6_Cohens_d,430AV2_ZT12_B6_J,430AV2_ZT12_B6_Hedges_g,430AV2_ZT12_B6_Var_d,430AV2_ZT12_B6_Var_g,430AV2_ZT12_B6_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_ZT12_B6_Enrich',u'430AV2_ZT12_B6_Hedges_g', u'430AV2_ZT12_B6_Var_g', u'430AV2_ZT12_B6_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_ZT12_B6_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_ZT12_B6_Enrich,430AV2_ZT12_B6_Hedges_g,430AV2_ZT12_B6_Var_g,430AV2_ZT12_B6_SEg
MGI symbol,,,,
0610005C13Rik,0.183627,1.224697,0.700118,0.836731
0610008F07Rik,0.195467,3.780387,1.979263,1.406863
0610009B22Rik,0.211600,10.236610,11.028949,3.320986
0610009D07Rik,0.278143,2.275466,1.082213,1.036978
0610009O20Rik,-0.236013,-2.843560,1.358713,1.165639
0610010K14Rik,-0.147705,-1.965634,0.936502,0.967730
0610012G03Rik,0.093386,1.535079,0.904438,0.940119
0610031J06Rik,-0.308765,-2.740474,1.301150,1.140680
0610037L13Rik,0.026165,0.238262,0.555807,0.745525
